In [ ]:
# --- Interannual percent change from mean (poly_area_ha) + rate of change ---
#     Left y-axis: percent change from mean  (single line color)
#     Right y-axis: rate of change (m^2/day) as red points
#     Optionally cap plotted rate values (for clarity) WITHOUT changing raw data.

import arcpy, os, pandas as pd, numpy as np, matplotlib.pyplot as plt
from matplotlib.dates import AutoDateLocator, ConciseDateFormatter

# ================== TUNABLES ==================
GDB         = r"D:\GIS\REEF_ISLAND_RESEARCH.gdb"
DATE_FIELD  = "date"
AREA_FIELD  = "poly_area_ha"   # in hectares

SITE_ORDER  = [
    "Podang Podang Lompo (Built-Up Cay)",
    "Podang Podang Caddi (Pristine Cay)",
    "Kodingareng Keke (Small Cay)"
]
FC_TO_SITE  = {
    "Csat_update_podlompo_poly":  "Podang Podang Lompo (Built-Up Cay)",
    "Csat_update_podcaddi_poly":  "Podang Podang Caddi (Pristine Cay)",
    "Csat_update_kodkeke_poly":   "Kodingareng Keke (Small Cay)",
}

# Figure geometry
FIG_WIDTH_IN, ROW_HEIGHT_IN = 12.0, 3.5

# Left-axis (percent change) styling
LINE_COLOR        = "#1f77b4"   # single line colour for all sites
RAW_LINE_WIDTH    = 1.6
RAW_ALPHA         = 0.85
TICK_SIZE_MAJOR   = 13
TICK_SIZE_MINOR   = 11
LABEL_SIZE        = 18
GRID_ALPHA        = 0.35
SHOW_BASELINE     = True
BASELINE_COLOR    = "#767676"
BASELINE_LW       = 1.2
BASELINE_ALPHA    = 0.9

# Right-axis (rate of change) styling
RATE_MARKER_SIZE   = 15          # scatter point size
RATE_COLOR         = "#d62728"   # red dots
RATE_ALPHA         = 0.9

# ---- Capping option for plotting only ----
CAP_RATE_FOR_PLOT  = True   # set False to use full range
RATE_CAP           = 800.0  # |rate| cap for plotting (m^2/day)

OUT_PNG = r"D:\GIS\plots\interannual_pct_and_rate_dual_axis_singleline_CAP.png"

# ---- OUTPUT PATHS FOR SUMMARY TABLES ----
OUT_PCT_SITE_CSV         = r"D:\GIS\stats\pct_change_summary_by_site.csv"
OUT_PCT_SITE_SEASON_CSV  = r"D:\GIS\stats\pct_change_summary_by_site_season.csv"
# ==============================================

arcpy.env.workspace = GDB


def _to_dt(x):
    if pd.isna(x):
        return pd.NaT
    try:
        return pd.to_datetime(x)
    except Exception:
        for fmt in ("%Y-%m-%d", "%d/%m/%Y", "%d-%m-%Y", "%m/%d/%Y"):
            try:
                return pd.to_datetime(x, format=fmt)
            except Exception:
                pass
        return pd.to_datetime(x, errors="coerce")


def fc_to_df(fc_name, site_label):
    """Load one feature class to DataFrame with date, area(ha), and site."""
    rows = []
    with arcpy.da.SearchCursor(os.path.join(GDB, fc_name), [DATE_FIELD, AREA_FIELD]) as cur:
        for d, a in cur:
            rows.append((d, a))
    df = pd.DataFrame(rows, columns=[DATE_FIELD, AREA_FIELD]).dropna()
    df[DATE_FIELD] = df[DATE_FIELD].apply(_to_dt)
    df = df.dropna(subset=[DATE_FIELD, AREA_FIELD])

    # percent change from mean (area in ha)
    site_mean = pd.to_numeric(df[AREA_FIELD], errors="coerce").dropna().mean()
    if not np.isfinite(site_mean) or site_mean == 0:
        raise ValueError(f"Non-finite or zero mean area for {fc_name}")

    df["pct"] = 100.0 * ((df[AREA_FIELD].astype(float) - site_mean) / site_mean)
    df["site"] = site_label

    # area in m^2 for rate-of-change calculation
    df["area_m2"] = df[AREA_FIELD].astype(float) * 10000.0
    return df


# ---------- LOAD & PREP DATA ----------
frames = [fc_to_df(fc, FC_TO_SITE[fc]) for fc in FC_TO_SITE]
df = pd.concat(frames, ignore_index=True)

# Categorical site ordering
df["site"] = pd.Categorical(df["site"], categories=SITE_ORDER, ordered=True)
df.sort_values(["site", DATE_FIELD], inplace=True)

# ---- RATE OF CHANGE (m^2/day) ----
df["dt_days"] = df.groupby("site")[DATE_FIELD].diff().dt.total_seconds() / 86400.0
df["dA_m2"]   = df.groupby("site")["area_m2"].diff()

# Avoid zero or negative time differences
mask_bad = df["dt_days"] <= 0
df.loc[mask_bad, ["dt_days", "dA_m2"]] = np.nan

# RAW rate (kept intact for any analysis)
df["rate_m2_day"] = df["dA_m2"] / df["dt_days"]

# ---- PLOTTING rate (can be capped) ----
if CAP_RATE_FOR_PLOT:
    df["rate_plot"] = df["rate_m2_day"].clip(lower=-RATE_CAP, upper=RATE_CAP)
else:
    df["rate_plot"] = df["rate_m2_day"]


# =====================================================================
# ========== NEW PART: QUANTITATIVE SUMMARY OF PERCENT CHANGE =========
# =====================================================================

def assign_season(ts):
    """Return DJF, MAM, JJA, SON based on month of a timestamp."""
    if pd.isna(ts):
        return np.nan
    m = ts.month
    if m in (12, 1, 2):
        return "DJF"
    elif m in (3, 4, 5):
        return "MAM"
    elif m in (6, 7, 8):
        return "JJA"
    else:
        return "SON"


df["season"] = df[DATE_FIELD].apply(assign_season)

def pct_summary(sub):
    """Return a Series of useful stats for 'pct' for one group."""
    vals = sub["pct"].astype(float).to_numpy()
    vals = vals[np.isfinite(vals)]
    if vals.size == 0:
        return pd.Series({
            "n":        0,
            "mean":     np.nan,
            "mean_abs": np.nan,
            "median":   np.nan,
            "std":      np.nan,
            "min":      np.nan,
            "p10":      np.nan,
            "p25":      np.nan,
            "p50":      np.nan,
            "p75":      np.nan,
            "p90":      np.nan,
            "max":      np.nan,
            "p90_abs":  np.nan,
        })
    abs_vals = np.abs(vals)
    return pd.Series({
        "n":        vals.size,
        "mean":     np.nanmean(vals),
        "mean_abs": np.nanmean(abs_vals),
        "median":   np.nanmedian(vals),
        "std":      np.nanstd(vals, ddof=1),
        "min":      np.nanmin(vals),
        "p10":      np.nanpercentile(vals, 10),
        "p25":      np.nanpercentile(vals, 25),
        "p50":      np.nanpercentile(vals, 50),
        "p75":      np.nanpercentile(vals, 75),
        "p90":      np.nanpercentile(vals, 90),
        "max":      np.nanmax(vals),
        "p90_abs":  np.nanpercentile(abs_vals, 90),
    })

# ---- (1) Summary by site ----
pct_by_site = (
    df.groupby("site", observed=True)
      .apply(pct_summary)
      .reset_index()
)

os.makedirs(os.path.dirname(OUT_PCT_SITE_CSV), exist_ok=True)
pct_by_site.to_csv(OUT_PCT_SITE_CSV, index=False)
print("Saved percent-change summary by site:", OUT_PCT_SITE_CSV)

# ---- (2) Summary by site & season ----
pct_by_site_season = (
    df.groupby(["site", "season"], observed=True)
      .apply(pct_summary)
      .reset_index()
)

pct_by_site_season.to_csv(OUT_PCT_SITE_SEASON_CSV, index=False)
print("Saved percent-change summary by site & season:", OUT_PCT_SITE_SEASON_CSV)

# =====================================================================
# =================== END OF NEW SUMMARY BLOCK =======================
# =====================================================================


# ---------- Y-LIMITS ----------
# Left axis (percent change) from all sites
all_vals = df["pct"].to_numpy()
vmin, vmax = np.nanmin(all_vals), np.nanmax(all_vals)
if not np.isfinite(vmin) or not np.isfinite(vmax):
    vmin, vmax = -20.0, 20.0
pad = 0.04 * (vmax - vmin if vmax > vmin else 40.0)
ymin_pct, ymax_pct = vmin - pad, vmax + pad

# Right axis (rate of change) from all sites, based on rate_plot
rate_vals_plot = df["rate_plot"].to_numpy()
rate_vals_plot = rate_vals_plot[np.isfinite(rate_vals_plot)]

if rate_vals_plot.size > 0:
    rmin, rmax = rate_vals_plot.min(), rate_vals_plot.max()
    # symmetric limits around zero
    rabs = max(abs(rmin), abs(rmax))
    rpad = 0.05 * rabs
    ymin_rate, ymax_rate = -(rabs + rpad), (rabs + rpad)
else:
    ymin_rate, ymax_rate = -10.0, 10.0

# ---------- PLOT ----------
fig, axes = plt.subplots(
    nrows=len(SITE_ORDER),
    ncols=1,
    figsize=(FIG_WIDTH_IN, ROW_HEIGHT_IN * len(SITE_ORDER)),
    sharex=True,
    sharey=True
)

if len(SITE_ORDER) == 1:
    axes = [axes]

date_loc = AutoDateLocator()
date_fmt = ConciseDateFormatter(date_loc)

panel_letters = ["a", "b", "c"]
right_axes = []

for ax_left, site, letter in zip(axes, SITE_ORDER, panel_letters):
    dsub = df[df["site"] == site].sort_values(DATE_FIELD)

    # --- RIGHT AXIS: rate of change (m^2/day) ---
    ax_right = ax_left.twinx()
    right_axes.append(ax_right)

    ax_right.scatter(dsub[DATE_FIELD], dsub["rate_plot"],
                     s=RATE_MARKER_SIZE, color=RATE_COLOR,
                     alpha=RATE_ALPHA, zorder=3)

    ax_right.set_ylim(ymin_rate, ymax_rate)
    ax_right.tick_params(axis="y", which="major",
                         labelsize=TICK_SIZE_MAJOR, width=1.5, length=7)
    ax_right.tick_params(axis="y", which="minor",
                         labelsize=TICK_SIZE_MINOR, width=1.2, length=4)

    # --- LEFT AXIS: percent change from mean ---
    if SHOW_BASELINE:
        ax_left.axhline(0.0, ls="--",
                        lw=BASELINE_LW,
                        color=BASELINE_COLOR,
                        alpha=BASELINE_ALPHA, zorder=0)

    ax_left.plot(dsub[DATE_FIELD], dsub["pct"],
                 color=LINE_COLOR, linewidth=RAW_LINE_WIDTH, alpha=RAW_ALPHA)

    ax_left.set_ylim(ymin_pct, ymax_pct)
    ax_left.set_title(site, loc="left", fontsize=14, weight="bold", color="black")
    ax_left.yaxis.grid(True, linestyle=":", alpha=GRID_ALPHA)

    ax_left.tick_params(axis="x", which="major",
                        labelsize=TICK_SIZE_MAJOR, width=1.5, length=7)
    ax_left.tick_params(axis="y", which="major",
                        labelsize=TICK_SIZE_MAJOR, width=1.5, length=7)
    ax_left.tick_params(axis="both", which="minor",
                        labelsize=TICK_SIZE_MINOR, width=1.2, length=4)

    ax_left.xaxis.set_major_locator(date_loc)
    ax_left.xaxis.set_major_formatter(date_fmt)

    # Panel letter in top-right
    ax_left.text(0.98, 0.92, letter,
                 transform=ax_left.transAxes,
                 ha="right", va="top",
                 fontsize=14, fontweight="bold", color="black")

# --------- AXIS LABELS (use middle & bottom panels) ----------
mid_idx = len(axes) // 2

axes[mid_idx].set_ylabel("Percent area change from mean (%)",
                         fontsize=LABEL_SIZE)
axes[mid_idx].yaxis.labelpad = 10

right_axes[mid_idx].set_ylabel("Rate of change (m$^2$/day)",
                               fontsize=LABEL_SIZE, rotation=270)
right_axes[mid_idx].yaxis.labelpad = 25

axes[-1].set_xlabel("Time", fontsize=LABEL_SIZE)
axes[-1].xaxis.labelpad = 12

# --------- LEGEND (top panel only) ----------
line_handle = axes[0].plot([], [], color=LINE_COLOR, linewidth=RAW_LINE_WIDTH,
                           label="Percent area change")[0]
dot_handle = axes[0].scatter([], [], s=RATE_MARKER_SIZE, color=RATE_COLOR,
                             alpha=RATE_ALPHA, label="Rate of change")

axes[0].legend(handles=[line_handle, dot_handle],
               frameon=False, fontsize=10, loc="upper left")

# Clean spacing
fig.subplots_adjust(left=0.16, right=0.86,
                    bottom=0.10, top=0.95,
                    hspace=0.10)

os.makedirs(os.path.dirname(OUT_PNG), exist_ok=True)
fig.savefig(OUT_PNG, dpi=300)
plt.close(fig)
print(f"Saved: {OUT_PNG}")
